# 07 — Types of Chains (Core Runnables)

In LangChain LCEL (LangChain Expression Language), **chains** are built by connecting *runnables* — small modular units that process data.

In this notebook, you’ll learn three fundamental types of runnables:

1. `RunnablePassthrough` — passes input unchanged.
2. `RunnableLambda` — wraps any custom Python function.
3. `RunnableParallel` — runs several branches in parallel and merges results.

In [1]:
# ╔══════════════════════════════════════════════════════╗
# ║ Setup: Load environment variables & initialize model ║
# ╚══════════════════════════════════════════════════════╝

import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI
# from langchain_groq import ChatGroq

chat_model = ChatOpenAI(model="gpt-4o-mini")
# chat_model = ChatGroq(model="llama-3.1-70b-versatile")

print("✅ Environment loaded and model ready.")

✅ Environment loaded and model ready.


# **1️⃣ RunnablePassthrough (LCEL)**

### What is it?
`RunnablePassthrough` returns exactly the same input it receives — without modification.
It’s useful when you want to **propagate the original input** along with other operations in a chain.

In [2]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()
print(chain.invoke("Abram"))  # → "Abram"

Abram


### 🧠 Practical uses

**1. Parallel branches (fan-out)**

When you want to construct a dictionary with multiple outputs, and one of them is the unmodified input:


In [3]:
pipeline = {
    "original": RunnablePassthrough(),
    "upper": lambda x: x.upper(),
    "len": lambda x: len(x),
}
print(pipeline["len"]("Abram"))  # → 5

5


**2. Combine context + input**

Keep the raw query while adding metadata:


In [ ]:
enrich = {
    "query": RunnablePassthrough(),
    "metadata": lambda q: {"chars": len(q), "has_space": " " in q},
}
print(enrich["metadata"]("LangChain rocks!"))

✅ **When to use**
- When you need to pass input untouched to the next stage.
- When building dict outputs with original + transformed versions.
- As a connector node to retain the original input.

⚠️ **Gotcha:** it passes references, not deep copies — if you mutate objects later, make your own copy first.

# **2️⃣ RunnableLambda (LCEL)**

### What is it?
`RunnableLambda` wraps any Python function and turns it into a runnable that can be composed inside a chain.
It’s the wildcard tool for custom logic.

In [4]:
from langchain_core.runnables import RunnableLambda

def russian_lastname(name: str) -> str:
    return f"{name}ovich"

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)
print(chain.invoke("Abram"))  # → "Abramovich"

Abramovich


### 🧠 Practical uses

1. **Inject custom logic**
```python
RunnableLambda(lambda x: x[::-1])  # reverses strings
```

2. **Preprocess data before LLM**
```python
def normalize(text: str) -> str:
    return text.lower().strip()
chain = RunnableLambda(normalize)
```

3. **Postprocess model results**
```python
clean_chain = chat_model | RunnableLambda(lambda x: x.replace("\n", " "))
```

4. **Integrate external libraries**
```python
import math
chain = RunnableLambda(lambda x: math.sqrt(x))
print(chain.invoke(9))  # → 3.0
```

✅ **When to use**
- For lightweight custom transformations.
- When a dedicated runnable doesn’t exist.
- When adding Python logic inline inside a chain.

⚠️ **Gotchas**
- Functions should be pure (no global side effects).
- Async functions are supported.
- Use `functools.partial` if you need fixed arguments.

# **3️⃣ RunnableParallel (LCEL)**

### What is it?
`RunnableParallel` executes several sub-tasks in parallel and merges the results into a single dictionary.

In LCEL, when you pass a **dict** in a chain, it automatically acts as a `RunnableParallel` — you don’t always need to instantiate it explicitly.

In [5]:
from langchain_core.runnables import RunnableParallel

def to_upper(text: str) -> str:
    return text.upper()

def count_chars(text: str) -> int:
    return len(text)

parallel_chain = RunnableParallel({
    "original": RunnablePassthrough(),
    "upper": RunnableLambda(to_upper),
    "chars": RunnableLambda(count_chars),
})

print(parallel_chain.invoke("LangChain"))

{'original': 'LangChain', 'upper': 'LANGCHAIN', 'chars': 9}


### 🧩 Explanation

Each branch runs independently and receives the same input.
When all finish, the results are merged into a dictionary.

👉 Output example:

```python
{'original': 'LangChain', 'upper': 'LANGCHAIN', 'chars': 9}
```

### 🧠 What is `itemgetter`?

`itemgetter` (from Python’s `operator` module) lets you extract a specific key from a dictionary.
It’s often used inside parallel or composed chains when you want to **propagate only part of the input**.

Example:

```python
from operator import itemgetter

extract_name = itemgetter('name')
print(extract_name({'name': 'Marie', 'age': 35}))  # → 'Marie'
```

This is useful when you want to route only a certain field into a chain stage.

### **Important: the syntax of RunnableParallel can have several variations**

When composing a `RunnableParallel` with another runnable, you don’t need to explicitly wrap it inside the class.

Inside a chain, the following three syntaxes are **equivalent**:

```python
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# 1️⃣ Explicit instantiation
RunnableParallel({"context": retriever, "question": RunnablePassthrough()})

# 2️⃣ Keyword-style instantiation
RunnableParallel(context=retriever, question=RunnablePassthrough())

# 3️⃣ Dictionary shorthand (most common in LCEL)
{"context": retriever, "question": RunnablePassthrough()}
```

In LCEL, passing a dictionary directly is the most typical way to express parallel branches — it’s automatically interpreted as a `RunnableParallel`.

## ✅ Summary

| Runnable Type | Purpose | Typical Use |
|----------------|----------|-------------|
| **RunnablePassthrough** | Pass input unchanged | Keep or reuse raw data |
| **RunnableLambda** | Wrap custom Python logic | Transformations, preprocessing, postprocessing |
| **RunnableParallel** | Run multiple branches at once | Build dicts, prepare multiple variables, fan-out inputs |

These three are the **foundation of modern LCEL chains**, allowing you to build modular, parallel, and composable workflows.